In [3]:
import numpy as np

In [17]:
import pandas as pd
import torch
import numpy as np
from sklearn import tree
from sklearn.metrics import classification_report, plot_confusion_matrix
import pydotplus


from imblearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours
from IPython.display import Image

In [18]:
diabetes_df = pd.read_csv("A:\launch_code_STL\Final_Homework\week-13\diabetes.csv")
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = diabetes_df.drop('Outcome', axis=1).values 
y = diabetes_df['Outcome'].values 


X_train, X_test, y_train, y_test =train_test_split(X, y, test_size = 0.3, random_state=42, stratify=y)

sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
print(X_train)
print(y_train)

[[ 0.93138344  2.0179454   0.78066953 ...  0.43148259 -0.37477883
   0.63212912]
 [ 0.63260632 -1.14861888  0.46538785 ... -0.1198324  -0.29416766
   0.71699246]
 [-0.56250219 -0.47692343 -0.2702694  ... -0.20958135  2.74517192
   0.03808578]
 ...
 [-0.86127931 -0.76479291  0.04501228 ...  0.76483585 -0.78380586
  -0.30136756]
 [ 0.63260632  2.20985838  1.2010451  ...  0.43148259 -0.60466993
   2.75371249]
 [ 0.03505207  0.73852549 -0.58555107 ... -0.33779414 -0.57779954
   0.29267578]]
[1 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 1 0 1 1
 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0
 0 0 1 0 0 0 1 0 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 1 1
 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 1 1 1 1 1 1 1
 0 0 1 1 0 0 0 0 0 1 1 1 1 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0
 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 0 1 0 0 1 1 1 0 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 1

In [35]:
import torch.nn as nn
import torch.nn.functional as F


# create Tensors = matrices

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(X_train)

tensor([[ 0.9314,  2.0179,  0.7807,  ...,  0.4315, -0.3748,  0.6321],
        [ 0.6326, -1.1486,  0.4654,  ..., -0.1198, -0.2942,  0.7170],
        [-0.5625, -0.4769, -0.2703,  ..., -0.2096,  2.7452,  0.0381],
        ...,
        [-0.8613, -0.7648,  0.0450,  ...,  0.7648, -0.7838, -0.3014],
        [ 0.6326,  2.2099,  1.2010,  ...,  0.4315, -0.6047,  2.7537],
        [ 0.0351,  0.7385, -0.5856,  ..., -0.3378, -0.5778,  0.2927]])


In [36]:
# Artificaial Neural Network

class ANN_Model(nn.Module):
    
    #uses the parameters for nn.Module, check documentation
    def __init__(self, input_features=8, hidden1=20, hidden2=20, out_features=2):
        
        # keyword super is a computed indirect reference, 
        # iolates changes and makes sure children in the layers of multiple inheritance
        # are calling the correct parents
        super().__init__() 
        
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
        
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x

In [37]:
torch.manual_seed(42)

#create instance of model
ann = ANN_Model()

In [39]:
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = ann.forward(X_train)
    loss = loss_function(y_pred, y_train)
    final_loss.append(loss)
    
    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss}')
        
    optimizer.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() 
    optimizer.step() #perform one optimization step each epoch

NameError: name 'loss_function' is not defined